# **Extracción de Información de los Proyectos de Trabajo de Grado del PCAM**

En este documento se describe el proceso de extracción de información de los  proyectos de trabajo de grado del Programa Curricular de Artes Musicales (PCAM) de la Academia Superior de Artes de Bogotá (ASAB), perteneciente a la Universidad Distrital Francisco José de Caldas.

La extracción consistirá en obtener datos clave como el nombre del estudiante, el código del proyecto, el título del trabajo, la pregunta problema planteada y la mención de la línea de investigación. Estos datos serán almacenados en una base de datos para su posterior análisis y utilización en el ámbito académico.

El objetivo es aprovechar las técnicas de Procesamiento de Lenguaje Natural (NLP) para automatizar la extracción de información y facilitar el estudio y la comprensión de los proyectos de trabajo de grado en la ASAB.

### **Primer Parte: Configuración de las Herramientas a usar y Conexión a las carpetas de Drive**

In [ ]:
# Importar las librerias o dependencias,
from google.colab import drive
import os
from openpyxl import Workbook

In [ ]:
# Descargar e instalar el modelo spaCy en español
!python -m spacy download es_core_news_sm

# Instalar la biblioteca python-docx
!pip install python-docx

# Importar la biblioteca spacy
import spacy

# Importar la biblioteca PyMuPDF para .pdf
!pip install PyMuPDF

In [ ]:
# Montar la unidad de Google Drive
drive.mount('/content/drive')

# Ruta de la carpeta raíz
carpeta_raiz = '/content/drive/MyDrive/Asab/TrabajoDeGradoHNavarro'

# Crear una nueva carpeta
carpeta_nueva = 'ResultadosCuatro'
os.makedirs(os.path.join(carpeta_raiz, carpeta_nueva))

# Crear un archivo de Excel
ruta_excel = os.path.join(carpeta_raiz, carpeta_nueva, 'resultados.xlsx')
wb = Workbook()
ws = wb.active

# Agregar encabezados
ws['A1'] = 'id'
ws['B1'] = 'nombre'
ws['C1'] = 'código'
ws['D1'] = 'titulo'
ws['E1'] = 'pregunta_problema'
ws['F1'] = 'linea_investigacion'
ws['G1'] = 'fecha'


# Guardar el archivo de Excel
wb.save(ruta_excel)

# Imprimir un mensaje de confirmación
print(f"Se ha creado la carpeta '{carpeta_nueva}' y el archivo de Excel '{ruta_excel}'")

### **Segunda Parte: Extracción del Titulo, Nombre, Código del Estudiante, Línea de Facultad, Línea de Proyecto, Pregunta Problema, Parrafo de Línea**

In [ ]:
import os
import pandas as pd
from docx import Document
import spacy
from spacy.matcher import Matcher
from IPython.display import display, HTML
import unicodedata
import re

# Crear una lista para almacenar los datos de cada documento
data = []

# Cargar el modelo de spaCy
nlp = spacy.load('es_core_news_sm')

# Crear un objeto Matcher
matcher = Matcher(nlp.vocab)

# Patrones a buscar
patron_linea_investigacion = [{"LOWER": "línea"}, {"LOWER": "de"}, {"LOWER": "investigación"}]
patron_lineas_investigacion = [{"LOWER": "líneas"}, {"LOWER": "de"}, {"LOWER": "investigación"}]
patron_linea_investigacion_sin_tilde = [{"LOWER": "linea"}, {"LOWER": "de"}, {"LOWER": "investigacion"}]
patron_lineas_investigacion_sin_tilde = [{"LOWER": "lineas"}, {"LOWER": "de"}, {"LOWER": "investigacion"}]

# Añadir los patrones al Matcher
matcher.add("LineaInvestigacion", [patron_linea_investigacion, patron_lineas_investigacion, patron_linea_investigacion_sin_tilde, patron_lineas_investigacion_sin_tilde])

# Definir la ruta de la carpeta raíz
ruta_carpeta = '/content/drive/MyDrive/Asab/TrabajoDeGradoHNavarro/PTGparaH/'

# Iterar sobre cada subcarpeta en la carpeta raíz
for subcarpeta in os.listdir(ruta_carpeta):
    # Crear la ruta completa de la subcarpeta
    ruta_subcarpeta = os.path.join(ruta_carpeta, subcarpeta)

    # Verificar si la subcarpeta es realmente una carpeta
    if os.path.isdir(ruta_subcarpeta):
        # Iterar sobre cada archivo en la subcarpeta
        for archivo in os.listdir(ruta_subcarpeta):
            # Verificar si el archivo es un documento .docx
            if archivo.endswith(".docx"):
                # Crear la ruta completa del documento
                ruta_documento = os.path.join(ruta_subcarpeta, archivo)

                # Leer el texto del documento
                try:
                    documento = Document(ruta_documento)
                    texto_docx = " ".join([p.text for p in documento.paragraphs])
                except Exception as e:
                    print(f"No se pudo leer el archivo {archivo}. Error: {e}")
                    continue

                # Convertir el texto a minúsculas y quitar tildes
                texto_docx = unicodedata.normalize('NFKD', texto_docx.lower()).encode('ASCII', 'ignore').decode('utf-8')

                # Procesar el texto con spaCy
                doc = nlp(texto_docx)

                # Encontrar coincidencias con los patrones de línea de investigación
                matches_linea_investigacion = matcher(doc)

                # Asignar la línea de investigación si se encontró
                if matches_linea_investigacion:
                    linea_investigacion = " "
                    for match_id, start, end in matches_linea_investigacion:
                        span = doc[start:end]
                        linea_investigacion += span.sent.text + "\n"
                else:
                    linea_investigacion = " "

                # Reemplazar los caracteres de nueva línea en la línea de investigación
                linea_investigacion = linea_investigacion.replace('\n', ' ')

                # Buscar las coincidencias directamente en el documento para la pregunta problema
                matches_pregunta_problema = [sent for sent in doc.sents if "pregunta problema" in sent.text.lower() or "preguntas problema" in sent.text.lower()]

                # Asignar la pregunta problema si se encontró
                if matches_pregunta_problema:
                    pregunta_problema = ""
                    for match in matches_pregunta_problema:
                        pregunta_problema += match.text + "\n"
                else:
                    pregunta_problema = " "

                # Eliminar "PREGUNTA PROBLEMA" del inicio de la pregunta problema
                pregunta_problema = pregunta_problema.split("PREGUNTA PROBLEMA", 1)[-1].split("Pregunta Problema", 1)[-1].strip()

                # Leer solo las primeras dos páginas del documento
                primeros_tres_parrafos = []

                for i, paragraph in enumerate(documento.paragraphs):
                    if i < 20:  # 20 párrafos aproximadamente, considerando que cada página tiene alrededor de 10 párrafos
                        texto = paragraph.text.strip()
                        if "UNIVERSIDAD DISTRITAL" not in texto and "FACULTAD DE ARTES ASAB" not in texto and "PROYECTO CURRICULAR" not in texto and "ÉNFASIS" not in texto:
                            primeros_tres_parrafos.append(texto)

                # Asignar el título y el nombre del estudiante según la longitud de palabras en el párrafo
                titulo = ""
                nombre_estudiante = ""
                codigo_estudiante = ""
                anio = ""
                for parrafo in primeros_tres_parrafos:
                    palabras = parrafo.split()
                    if len(palabras) > 4:  # Si el párrafo tiene más de 4 palabras, se considera el título
                        titulo = parrafo
                    elif re.search(r"([A-Za-z]+\s[A-Za-z]+\s[A-Za-z]+\s?[A-Za-z]+?)", parrafo):  # Patrón para identificar nombres en mayúsculas y minúsculas con máximo 4 palabras
                        nombre_estudiante = parrafo

                    # Buscar el código del estudiante en el párrafo
                    codigo_pattern = r"\d{11}"  # Patrón para encontrar un número de 11 dígitos
                    codigos_encontrados = re.findall(codigo_pattern, parrafo)
                    if codigos_encontrados:
                        codigo_estudiante = codigos_encontrados[0]

                    # Buscar el año en el párrafo
                    anio_pattern = r"\b(20\d{2})\b"  # Patrón para encontrar un número de 4 dígitos que comienza con "20"
                    anios_encontrados = re.findall(anio_pattern, parrafo)
                    if anios_encontrados:
                        anio = anios_encontrados[0]

                # Buscar las líneas de facultad y proyecto curricular
                lineas_facultad = ["arte y sociedad", "estetica y teorias", "arte y pedagogia", "arte y culturas"]
                lineas_proyecto = ["percepcion, experiencia", "musica y contexto", "pedagogia y didactica", "agentes culturales", "musica, ciencia", "pedagogia y didacticas", "creacion musical","analisis para la","estudios del campo"]

                linea_facultad_encontrada = ""
                linea_proyecto_encontrada = ""

                for linea_facultad in lineas_facultad:
                    if linea_facultad in texto_docx.lower():
                        linea_facultad_encontrada = linea_facultad
                        break

                for linea_proyecto in lineas_proyecto:
                    if linea_proyecto in texto_docx.lower():
                        linea_proyecto_encontrada = linea_proyecto
                        break

                # Agregar los datos del documento actual a la lista
                data.append({"documento": archivo, "código del estudiante": codigo_estudiante, "nombre del estudiante": nombre_estudiante, "título": titulo, "año": anio, "línea de facultad": linea_facultad_encontrada, "línea de proyecto": linea_proyecto_encontrada, "línea de investigación": linea_investigacion, "pregunta problema": pregunta_problema})

# Crear un DataFrame con los datos
df = pd.DataFrame(data)

display(HTML(df.to_html()))

## **Tercera Parte: Almacenar el Data Frame en el Documento de Excel**

In [ ]:
# Definir la carpeta raíz
carpeta_raiz = '/content/drive/MyDrive/Asab/TrabajoDeGradoHNavarro'

# Definir la ruta del archivo de Excel
ruta_excel = os.path.join(carpeta_raiz, 'preguntas.xlsx')

# Guardar el DataFrame en un archivo de Excel
df.to_excel(ruta_excel, index=False)

print(f"Los resultados se han guardado con éxito en el archivo de Excel en {ruta_excel}")
